In [ ]:
import os
import tempfile
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from dotenv import load_dotenv
import streamlit as st
import uuid

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.schema import Document
from langchain.chains import LLMRouterChain, MultiRetrievalQAChain, RetrievalQA
from langchain.retrievers import MultiVectorRetriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
#오픈AI API 키 설정
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

## 1. 문서 불러오기

In [3]:
import tempfile

In [4]:
file_path = '../data/langchainThon/cv/경력기술서_천승우.docx'
# 파일을 바이너리 모드로 읽어서 _file처럼 흉내내기
class DummyFile:
    def __init__(self, path):
        self.name = path
        with open(path, "rb") as f:
            self._data = f.read()
    def getvalue(self):
        return self._data

_file = DummyFile(file_path)

In [5]:
def load_cv(_file):
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as tmp_file:
        tmp_file.write(_file.getvalue())
        tmp_file_path = tmp_file.name
        
    #PDF 파일 업로드
    if _file.name.endswith('.pdf'):
        loader = PyPDFLoader(file_path=tmp_file_path)
    
    #Word 파일 업로드
    elif _file.name.endswith('.docx'):
        loader = Docx2txtLoader(file_path=tmp_file_path)
    
    # 파일 형식이 틀릴경우 에러 메세지 출력
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 업로드해주세요.")
    
    pages = loader.load()
    return pages

In [6]:
cv_pages = load_cv(_file)
cv_pages

[Document(metadata={'source': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpx1ivf2l6'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. \n\n\n\n[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크\n\npandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle\n\n운영체제\n\nLinux(Ubuntu), Windows, macOS\n\nPalantir Foundry\n\nPipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset\n\n기타\n\nVim / Vi, Git / GitHub, Bioinformatics Tools (Gene Set Enrichment Analysis, Bismark, Metilene, lifelines, survival, survminer, lifelines)\n

In [7]:
def load_jd(_url: str):
    loader = WebBaseLoader(_url)
    loader.requests_kwargs = {'verify': False}  # SSL 검증 비활성화
    return loader.load()

In [8]:
# jd pages
jd_pages = load_jd('https://www.wanted.co.kr/wd/297589')
jd_pages

[Document(metadata={'source': 'https://www.wanted.co.kr/wd/297589', 'title': '[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드', 'description': '임팩티브에이아이의 [Deepflow] 데이터 사이언티스트 (Data Scientist) 포지션을 확인해 보세요. 취업·이직에 성공하면, 합격보상금 50만원을 드립니다.\n포지션: [Deepflow] 데이터 사이언티스트 (Data Scientist)\n회사 위치: 서울 강남구\n자격 요건: • 시계열 예측 모델 및 시스템 구축 프로젝트 경험자\n• 인공지능, 데이터과학, 전산학, 통계 등 관련 전공 석사학위 이상\n• 머신러닝 및 데이터 분석 분야에서 5년 이상 경력 보유하신 분\n주요 업무: 임팩티브AI의 데이터사이언티스트는 머신러닝/딥러닝 모델 구축, 시계열 예측, 데이터분석 업무를 수행하게 됩니다. \n소프트웨어 엔지니어, 서비스 기획자, 도메인 전문가와 협력하여, 정확하고 확장가능한 예측모델을 개발 및 최적화하는 역할을 담당하게 됩니다.\n\n  • 수요예측 위한 데이터 분석 및 예측모델 설계/개발\n  • 데이터 전처리 및 피처 엔지니어링\n  • 인과 추론 및 설명가능성 구현을 위한 분석\n  • 고객사 딜리버리 위한 예측모델 구축 및 배포\n  • 최신 모델 및 데이터 분석 트렌드 연구 및 적용', 'language': 'ko-KR'}, page_content='[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드채용이력서교육•이벤트콘텐츠소셜프리랜서더보기회원가입/로그인회원가입기업 서비스1/6임팩티브에이아이∙서울 강남구∙경력 5-12년[Deepflow] 데이터 사이언티스트 (Data Scientist)포지션 상세[임팩티브AI 소개]임팩티브AI는 수요예측에 특화된 AI 솔루션 전문기업입니다. 자체 솔루션 Deepflow

## 2. 텍스트 나누기, 임베딩, 벡터DB 생성

In [9]:
def chunk_documents(pages, source_label, chunk_size=500, chunk_overlap=100):
    '''
    페이지(Document list)들을 청크로 나누고 metadata를 추가하는 함수
    - pages: Document list
    - source_label: 'cv' 또는 'jd' 등 source 표시
    - return: 청크가 나뉜 Document 리스트
    '''
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n'],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False
    )

    split_docs = text_splitter.split_documents(pages)

    split_docs_with_meta = [
        Document(page_content=d.page_content,
                 metadata={'source': source_label, 'chunk_id': str(i)})
        for i, d in enumerate(split_docs)
    ]

    return split_docs_with_meta

In [10]:
def create_vector_store(_cv, _jd):
    '''
    CV, JD 문서를 Chroma 벡터스토어로 변환
    - st.cache_resource 제거 → 반복 호출해도 누적 문제 없음
    '''
    # 청크 생성
    cv_docs = chunk_documents(_cv, 'cv')
    jd_docs = chunk_documents(_jd, 'jd')

    # OpenAI 임베딩 모델
    embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

    # 컬렉션 이름을 매번 새로 생성
    collection_name = f'vectorstore_{uuid.uuid4()}'

    # Chroma vectorstore 생성
    vectorstore = Chroma.from_documents(
        documents=cv_docs + jd_docs,
        embedding=embeddings_model,
        collection_name=collection_name,
        persist_directory=None  # 메모리 기반
    )

    return vectorstore

In [11]:
vectorstore = create_vector_store(_cv=load_cv(_file), _jd=load_jd('https://www.wanted.co.kr/wd/297589'))

In [12]:
cv_docs_count = len([
    d for d in vectorstore._collection.get(include=['metadatas', 'documents'])['metadatas']
    if d['source'] == 'cv'
])
cv_docs_count

11

In [13]:
jd_docs_count = len([
    d for d in vectorstore._collection.get(include=["metadatas", "documents"])['metadatas']
    if d['source'] == 'jd'
])
jd_docs_count

2

In [14]:
vectorstore._collection.count()

13

## 3. Retriever 구성 및 Chaining

In [ ]:
def chaining(_cv, _jd):
    vectorstore = create_vector_store(_cv, _jd)

    # CV/JD 문서 수
    cv_docs_count = len([d for d in vectorstore._collection.get(include=['metadatas'])['metadatas'] if d['source']=='cv'])
    jd_docs_count = len([d for d in vectorstore._collection.get(include=['metadatas'])['metadatas'] if d['source']=='jd'])

    # CV 검색기
    cv_retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 5, 'filter': {'source': 'cv'}}
    )

    # JD 검색기
    jd_retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 5, 'filter': {'source': 'jd'}}
    )

    # 두 문서 모두 검색기 (fallback)
    both_retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 5}
    )

    # QA 시스템 프롬프트
    qa_system_prompt = '''
    당신은 커리어 및 채용 관련 질의응답을 돕는 AI 어시스턴트입니다.  
    아래 제공된 문서들을 참고하여, 사용자의 질문에 정확하고 논리적으로 답변하세요.  

    - 사용자의 **경력, 경험, 이력 관련 질문**이라면 CV(이력서)를 우선적으로 참고하세요.  
    - **지원하는 회사, 직무, 채용 요건** 관련 질문이라면 JD(공고문)를 우선적으로 참고하세요.  
    - 질문의 맥락상 두 문서가 모두 관련되어 있다면, **균형 있게 통합하여 답변**하세요.  
    - 문서에 직접적인 정보가 없을 경우, 일반적인 HR/커리어 상식과 논리를 기반으로 보완해 설명할 수 있습니다.  

    **언어 스타일 지침**
    - 답변은 언제나 **전문적이고 신뢰감 있는 어투**로 작성합니다.  
    - 문장은 명료하고 간결하게 유지하되, 비즈니스 상황에 맞는 적절한 어휘를 사용합니다.  
    - 필요 시 가벼운 이모지를 활용하여 자연스럽게 친근함을 더할 수 있으나, 과도하게 사용하지 않습니다.  
    
    {context}
    '''

    qa_prompt = ChatPromptTemplate.from_messages([
        ('system', qa_system_prompt),
        ('human','{input}')
    ])

    # MultiRetrievalQAChain 구성
    retrievers = [
        {'name': 'cv', 'description': '사용자 경력 및 이전 회사 관련 질문', 'retriever': cv_retriever, 'prompt': qa_prompt},
        {'name': 'jd', 'description': '지원을 원하는 회사 및 직무기술서 관련 질문', 'retriever': jd_retriever, 'prompt': qa_prompt},
    ]

    llm = ChatOpenAI(model="gpt-4o-mini")

    rag_chain = MultiRetrievalQAChain.from_retrievers(
        llm=llm,
        retriever_infos=retriever_infos,
        default_retriever=both_retriever,  # 'both' fallback
    )

    return rag_chain

In [ ]:
rag_chain = chaining(_cv=load_cv(_file), _jd=load_jd('https://www.wanted.co.kr/wd/297589'))

In [35]:
# 입력 키 매핑을 명시적으로 해주는 wrapper
from langchain_core.runnables import RunnableLambda

fixed_rag_chain = RunnableLambda(lambda x: {"question": x["input"]}) | rag_chain
fixed_rag_chain.invoke({"input": "과거 다녔던 회사에 대해 설명해줘"})

ValueError: Missing some input keys: {'input'}

ValueError: Missing some input keys: {'input'}

In [19]:
def test_rag_chain(rag_chain):
    """
    정의된 rag_chain에 여러 질문을 던져 응답과 선택된 retriever를 테스트하는 함수
    """
    test_cases = [
        ("이전 경력(CV) 관련 질문", "내가 이전에 다녔던 회사명을 맞춰봐"),
        ("지원 직무(JD) 관련 질문", "지원하려는 직무의 필수 자격 요건은 무엇인가요?"),
        ("두 문서 통합(CV+JD) 관련 질문", "나의 경력 중 지원 직무와 가장 관련성이 높은 경험은 무엇이며, 그 이유는 무엇인가요?")
    ]
    
    for title, question in test_cases:
        print(f'--- {title} ---')
        print(f'[질문]: {question}')
        
        try:
            # 반드시 {'input': 질문} 형태로 전달
            response = rag_chain.invoke({"input": question})
            
            # MultiRetrievalQAChain의 출력 구조 확인
            # 기본적으로 'output_text' 키가 존재
            output_text = response.get("output_text", "응답을 찾을 수 없습니다.")
            used_retriever = response.get("destination", "정보 없음")  # router가 선택한 retriever
            
            print(f'[응답]: {output_text}')
            print(f'[선택된 리트리버]: {used_retriever}')
            print('-' * 50)
            
        except Exception as e:
            print(f'{title} 테스트 중 오류 발생: {e}')
            print('-' * 50)

    
    print('--- 3. 두 문서 통합(CV+JD) 관련 질문 테스트 ---')
    integrated_question = '나의 경력 중 지원 직무와 가장 관련성이 높은 경험은 무엇이며, 그 이유는 무엇인가요?'
    print(f'[질문]: {integrated_question}')
    
    try:
        # 'input' 키 및 올바른 들여쓰기 적용
        integrated_response = rag_chain.invoke(integrated_question)
        
        print(f'[응답]: {integrated_response.get("result", "응답을 찾을 수 없습니다.")}')
        print(f'[선택된 리트리버]: {integrated_response.get("source_chain", "정보 없음")}')
        print('-' * 50)
        
    except Exception as e:
        print(f'통합 질문 테스트 중 오류 발생: {e}')

In [20]:
test_rag_chain(rag_chain)

--- 이전 경력(CV) 관련 질문 ---
[질문]: 내가 이전에 다녔던 회사명을 맞춰봐
이전 경력(CV) 관련 질문 테스트 중 오류 발생: Missing some input keys: {'input'}
--------------------------------------------------
--- 지원 직무(JD) 관련 질문 ---
[질문]: 지원하려는 직무의 필수 자격 요건은 무엇인가요?
지원 직무(JD) 관련 질문 테스트 중 오류 발생: Missing some input keys: {'input'}
--------------------------------------------------
--- 두 문서 통합(CV+JD) 관련 질문 ---
[질문]: 나의 경력 중 지원 직무와 가장 관련성이 높은 경험은 무엇이며, 그 이유는 무엇인가요?
두 문서 통합(CV+JD) 관련 질문 테스트 중 오류 발생: Missing some input keys: {'input'}
--------------------------------------------------
--- 3. 두 문서 통합(CV+JD) 관련 질문 테스트 ---
[질문]: 나의 경력 중 지원 직무와 가장 관련성이 높은 경험은 무엇이며, 그 이유는 무엇인가요?
통합 질문 테스트 중 오류 발생: Missing some input keys: {'input'}


## 4. 검색 → LLM에 컨텍스트 전달 → 응답 받기

In [31]:
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=[
        {'role': 'system', 'content': '다음 정보를 기반으로 사용자가 다녔던 회사를 알려주세요.'},
        {'role': 'user', 'content': context + '\n\n질문: ' + query}
    ]
)

print(response.choices[0].message.content)

당신이 다녔던 회사는 "임팩티브에이아이"입니다. 이 회사는 AI 솔루션 전문기업으로, 제품 수요 예측, 원자재 가격 예측 및 신제품 성과 예측 등 다양한 분야에서 기술력을 인정받고 있습니다.
